This notebook illustrates how to use `schema.py` to construct a JSON catalog file from a set of parameter estimation samples and other metadata.

In [ ]:
import numpy as np
import pandas as pd
from ccverify import schema

In [ ]:
def make_dummy_parameter_set():
    """Return schema.ParameterSet with dummy parameter set."""
    link = schema.Link(
        url='https://example.com',
        content_type='posterior_samples',
        description='string')

    samples = pd.DataFrame(
        {'mass_1_source': np.random.normal(10, 1, 100),
         'luminosity_distance': np.random.normal(130, 10, 100)})

    parameter_set = schema.ParameterSet.from_samples(
        samples=samples,
        name='string',
        data_url='https://zenodo.org/',
        waveform_family='IMRPhenomPv3HM',
        links=[link]
        )
    return parameter_set


def make_dummy_event():
    """Return ``schema.Event`` with dummy event."""
    search_results = [schema.SearchResult(pipeline='pipeline_name',
                                          pastro=0.12,
                                          far=0.89)
                     ]
    pe_sets = [make_dummy_parameter_set()]
    return schema.Event(name='GW123456_123456',
                        gps=123456789.0,
                        detectors=['H1', 'L1', 'V1'],
                        search=search_results,
                        pe_sets=pe_sets,
                        description='string')


def make_dummy_catalog():
    """Return ``schema.Catalog`` with IAS catalog."""
    events = [make_dummy_event()]
    return schema.Catalog(name='catalog name',
                          description='catalog description',
                          doi='https://doi.org/12345/',
                          events=events)

In [31]:
catalog = make_dummy_catalog()

In [32]:
catalog.to_json('dummy_catalog.json')

In [33]:
!ccverify dummy_catalog.json

[2024-05-09 17:37:22,185][INFO] ccverify.core: Verification finalized.
